# FastAPI para Data Science - Bootcamp Completo
## Parte 2: Modelos Pydantic, Métodos HTTP y Request Body

---

## 1. Introducción a Pydantic

### ¿Qué es Pydantic?

Pydantic es una librería de Python para validación de datos y gestión de configuraciones usando anotaciones de tipo de Python. Es el corazón de FastAPI para validar y serializar datos.

### ¿Por qué Pydantic es importante?

1. **Validación automática**: Valida los datos según los tipos definidos
2. **Conversión de tipos**: Convierte automáticamente los datos al tipo correcto cuando es posible
3. **Mensajes de error claros**: Proporciona errores detallados cuando la validación falla
4. **Serialización**: Convierte objetos Python a JSON y viceversa
5. **IDE support**: Autocompletado y verificación de tipos en tiempo de desarrollo
6. **Documentación automática**: FastAPI usa los modelos Pydantic para generar la documentación

### BaseModel: La clase base de Pydantic

`BaseModel` es la clase de la que heredan todos los modelos de Pydantic. Proporciona toda la funcionalidad de validación y serialización.

In [ ]:
# Instalamos pydantic si no está instalado (debería venir con FastAPI)
!pip install pydantic

In [ ]:
# Ejemplo básico de un modelo Pydantic
from pydantic import BaseModel
from typing import Optional

# Definimos un modelo simple para un Item
class Item(BaseModel):
    """
    Modelo Pydantic para representar un Item.
    
    Attributes:
        name: Nombre del item (obligatorio, string)
        price: Precio del item (obligatorio, float)
        description: Descripción del item (opcional, string)
        tax: Impuesto aplicable (opcional, float)
    """
    name: str                    # Campo obligatorio de tipo string
    price: float                 # Campo obligatorio de tipo float
    description: Optional[str] = None  # Campo opcional con valor por defecto None
    tax: Optional[float] = None  # Campo opcional con valor por defecto None

# Creamos una instancia del modelo
# Pydantic validará automáticamente los tipos
item = Item(
    name="Laptop",
    price=999.99,
    description="Una laptop potente",
    tax=21.0
)

print("Item creado:")
print(item)
print()

# Convertimos el modelo a diccionario
print("Como diccionario:")
print(item.dict())
print()

# Convertimos el modelo a JSON
print("Como JSON:")
print(item.json())
print()

# Accedemos a los campos individuales
print(f"Nombre: {item.name}")
print(f"Precio: {item.price}")
print(f"Precio con impuesto: {item.price + (item.tax if item.tax else 0)}")

In [ ]:
# Veamos qué pasa cuando intentamos crear un item con datos incorrectos
from pydantic import ValidationError

try:
    # Intentamos crear un item sin el campo obligatorio 'name'
    item_incorrecto = Item(
        price=999.99
    )
except ValidationError as e:
    print("Error de validación (falta campo obligatorio):")
    print(e)
    print()

try:
    # Intentamos crear un item con tipo incorrecto
    # price debe ser float, pero pasamos un string no convertible
    item_incorrecto = Item(
        name="Laptop",
        price="no es un número"
    )
except ValidationError as e:
    print("Error de validación (tipo incorrecto):")
    print(e)
    print()

# Pydantic intenta convertir tipos cuando es posible
# Por ejemplo, puede convertir un string numérico a float
item_con_conversion = Item(
    name="Laptop",
    price="999.99",  # String, pero convertible a float
    tax="21"         # String, pero convertible a float
)
print("Item creado con conversión automática:")
print(item_con_conversion)
print(f"Tipo de price: {type(item_con_conversion.price)}")
print(f"Tipo de tax: {type(item_con_conversion.tax)}")

---
## 2. Validaciones Avanzadas con Pydantic

### Field: Añadiendo metadatos y validaciones

Pydantic proporciona `Field` para añadir validaciones y metadatos a los campos del modelo, similar a como `Query` funciona para parámetros de consulta.

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Optional, List
from datetime import datetime

# Modelo con validaciones usando Field
class Product(BaseModel):
    """
    Modelo de producto con validaciones avanzadas.
    """
    # Campo con validación de longitud
    name: str = Field(
        ...,                                    # ... indica que es obligatorio
        min_length=3,                           # Mínimo 3 caracteres
        max_length=100,                         # Máximo 100 caracteres
        description="Nombre del producto"
    )
    
    # Campo con validación de rango
    price: float = Field(
        ...,
        gt=0,                                   # Greater than: mayor que 0 (estricto)
        description="Precio del producto en euros"
    )
    
    # Campo opcional con valor por defecto
    discount: float = Field(
        0.0,
        ge=0,                                   # Mayor o igual a 0
        le=100,                                 # Menor o igual a 100
        description="Descuento en porcentaje (0-100)"
    )
    
    # Campo con lista
    tags: List[str] = Field(
        default=[],                             # Lista vacía por defecto
        description="Etiquetas del producto"
    )
    
    # Campo con ejemplo para la documentación
    sku: str = Field(
        ...,
        regex="^[A-Z]{3}-[0-9]{4}$",           # Formato: ABC-1234
        description="Código SKU del producto",
        example="LAP-1234"
    )
    
    # Campo de fecha
    created_at: datetime = Field(
        default_factory=datetime.now,           # Función que genera el valor por defecto
        description="Fecha de creación"
    )

# Probamos el modelo
product = Product(
    name="Laptop Gaming",
    price=1299.99,
    discount=15.0,
    tags=["gaming", "laptop", "electronics"],
    sku="LAP-1234"
)

print("Producto creado:")
print(product.json(indent=2))  # indent=2 para formato legible

In [ ]:
# Validadores personalizados con @validator
from pydantic import BaseModel, Field, validator
from typing import List

class User(BaseModel):
    """
    Modelo de usuario con validadores personalizados.
    """
    username: str = Field(..., min_length=3, max_length=20)
    email: str = Field(..., description="Email del usuario")
    age: int = Field(..., ge=18, le=120, description="Edad del usuario")
    password: str = Field(..., min_length=8)
    password_confirm: str = Field(..., min_length=8)
    interests: List[str] = Field(default=[])
    
    # Validador para el email
    @validator('email')
    def validate_email(cls, v):
        """
        Valida que el email tenga formato correcto.
        
        Args:
            v: Valor del campo email
        
        Returns:
            str: Email validado en minúsculas
        
        Raises:
            ValueError: Si el email no tiene formato válido
        """
        # Validación simple de email (en producción usar regex más robusto o librería)
        if '@' not in v or '.' not in v:
            raise ValueError('Email debe contener @ y .')
        # Convertimos a minúsculas
        return v.lower()
    
    # Validador para el username
    @validator('username')
    def validate_username(cls, v):
        """
        Valida que el username solo contenga caracteres alfanuméricos y guiones bajos.
        """
        if not v.replace('_', '').isalnum():
            raise ValueError('Username solo puede contener letras, números y guiones bajos')
        return v
    
    # Validador que verifica que las contraseñas coincidan
    @validator('password_confirm')
    def passwords_match(cls, v, values):
        """
        Valida que password y password_confirm sean iguales.
        
        Args:
            v: Valor de password_confirm
            values: Diccionario con los valores de los campos ya validados
        """
        # 'password' estará en values si ya fue validado
        if 'password' in values and v != values['password']:
            raise ValueError('Las contraseñas no coinciden')
        return v
    
    # Validador para la lista de intereses
    @validator('interests')
    def validate_interests(cls, v):
        """
        Valida que no haya intereses duplicados.
        """
        if len(v) != len(set(v)):
            raise ValueError('Los intereses no pueden estar duplicados')
        return v

# Probamos el modelo User con validaciones correctas
print("Usuario válido:")
user = User(
    username="juan_perez",
    email="Juan.Perez@Example.COM",  # Se convertirá a minúsculas
    age=25,
    password="password123",
    password_confirm="password123",
    interests=["python", "data science", "AI"]
)
print(user.json(indent=2))
print()

In [ ]:
# Probamos validaciones que fallan
from pydantic import ValidationError

# Email inválido
try:
    user_invalid = User(
        username="juan_perez",
        email="email_sin_arroba",
        age=25,
        password="password123",
        password_confirm="password123"
    )
except ValidationError as e:
    print("Error: Email inválido")
    print(e)
    print()

# Contraseñas no coinciden
try:
    user_invalid = User(
        username="juan_perez",
        email="juan@example.com",
        age=25,
        password="password123",
        password_confirm="different_password"
    )
except ValidationError as e:
    print("Error: Contraseñas no coinciden")
    print(e)
    print()

# Username con caracteres inválidos
try:
    user_invalid = User(
        username="juan-perez!",  # Contiene guión y exclamación (no permitidos)
        email="juan@example.com",
        age=25,
        password="password123",
        password_confirm="password123"
    )
except ValidationError as e:
    print("Error: Username con caracteres inválidos")
    print(e)

---
## 3. Métodos HTTP en FastAPI

### Los métodos HTTP principales

HTTP define varios métodos (también llamados verbos) para interactuar con recursos:

1. **GET**: Obtener/leer datos. No debe modificar el servidor. Es idempotente (llamarlo múltiples veces produce el mismo resultado).
   - Ejemplo: Obtener una lista de productos, leer un usuario

2. **POST**: Crear nuevos recursos. Envía datos al servidor para crear algo nuevo.
   - Ejemplo: Crear un nuevo usuario, añadir un producto

3. **PUT**: Actualizar/reemplazar un recurso completo existente.
   - Ejemplo: Actualizar todos los datos de un usuario

4. **PATCH**: Actualizar parcialmente un recurso existente.
   - Ejemplo: Cambiar solo el email de un usuario

5. **DELETE**: Eliminar un recurso.
   - Ejemplo: Borrar un producto, eliminar un usuario

### Decoradores en FastAPI

FastAPI proporciona decoradores para cada método HTTP:
- `@app.get()`
- `@app.post()`
- `@app.put()`
- `@app.patch()`
- `@app.delete()`

---
## 4. Request Body con POST

### ¿Qué es el Request Body?

El Request Body (cuerpo de la petición) es la parte de una petición HTTP que contiene los datos enviados al servidor. Se usa principalmente con POST, PUT y PATCH.

A diferencia de los query parameters que van en la URL, el request body puede contener estructuras de datos complejas (objetos, listas anidadas, etc.).

### Usando modelos Pydantic como Request Body

En FastAPI, definimos el request body usando modelos Pydantic como tipo de un parámetro de la función:

In [ ]:
# Creamos un archivo para nuestra API con métodos POST

with open('main_crud.py', 'w', encoding='utf-8') as f:
    f.write('''from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, Field, validator
from typing import Optional, List
from datetime import datetime

# Inicializamos la aplicación
app = FastAPI(
    title="API CRUD Completa",
    description="API con operaciones Create, Read, Update, Delete",
    version="2.0.0"
)

# ============================================
# MODELOS PYDANTIC
# ============================================

class ItemBase(BaseModel):
    """
    Modelo base para un Item.
    Contiene los campos que el usuario puede proporcionar.
    """
    name: str = Field(
        ...,
        min_length=1,
        max_length=100,
        description="Nombre del item"
    )
    description: Optional[str] = Field(
        None,
        max_length=500,
        description="Descripción del item"
    )
    price: float = Field(
        ...,
        gt=0,
        description="Precio del item"
    )
    tax: Optional[float] = Field(
        None,
        ge=0,
        le=100,
        description="Porcentaje de impuesto (0-100)"
    )
    tags: List[str] = Field(
        default=[],
        description="Etiquetas del item"
    )

class ItemCreate(ItemBase):
    """
    Modelo para crear un Item.
    Hereda de ItemBase.
    """
    pass  # De momento es igual a ItemBase

class Item(ItemBase):
    """
    Modelo completo de Item que incluye campos generados por el servidor.
    """
    id: int = Field(..., description="ID único del item")
    created_at: datetime = Field(
        default_factory=datetime.now,
        description="Fecha de creación"
    )
    
    class Config:
        # Configuración para que Pydantic genere ejemplos en la documentación
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Laptop",
                "description": "Una laptop potente para desarrollo",
                "price": 1299.99,
                "tax": 21.0,
                "tags": ["electronics", "computers"],
                "created_at": "2024-01-15T10:30:00"
            }
        }

# ============================================
# BASE DE DATOS SIMULADA (en memoria)
# ============================================

# En una aplicación real, esto sería una base de datos
# Lista para almacenar items
items_db: List[Item] = []

# Contador para IDs únicos
next_id = 1

# ============================================
# ENDPOINTS
# ============================================

@app.get("/")
def read_root():
    """Endpoint raíz con información de la API."""
    return {
        "mensaje": "API CRUD de Items",
        "version": "2.0.0",
        "endpoints": {
            "GET /items/": "Listar todos los items",
            "GET /items/{item_id}": "Obtener un item por ID",
            "POST /items/": "Crear un nuevo item",
            "PUT /items/{item_id}": "Actualizar un item completo",
            "DELETE /items/{item_id}": "Eliminar un item"
        }
    }

# CREATE - Crear un nuevo item
@app.post(
    "/items/",
    response_model=Item,                    # El modelo de respuesta
    status_code=status.HTTP_201_CREATED     # Código 201 para creación exitosa
)
def create_item(item: ItemCreate):
    """
    Crea un nuevo item.
    
    Args:
        item (ItemCreate): Datos del item a crear (en el request body)
    
    Returns:
        Item: El item creado con su ID y fecha de creación
    """
    global next_id
    
    # Creamos el item completo con ID y fecha
    new_item = Item(
        id=next_id,
        **item.dict(),                      # Desempaquetamos los datos de ItemCreate
        created_at=datetime.now()
    )
    
    # Añadimos a nuestra "base de datos"
    items_db.append(new_item)
    
    # Incrementamos el ID para el próximo item
    next_id += 1
    
    return new_item

# READ - Obtener todos los items
@app.get(
    "/items/",
    response_model=List[Item]               # Respuesta es una lista de Items
)
def read_items(
    skip: int = 0,
    limit: int = 10,
    tag: Optional[str] = None               # Filtro opcional por tag
):
    """
    Obtiene una lista de items con paginación y filtrado opcional.
    
    Args:
        skip: Número de items a saltar
        limit: Número máximo de items a retornar
        tag: Filtrar items que contengan este tag (opcional)
    
    Returns:
        List[Item]: Lista de items
    """
    # Si hay filtro por tag, aplicarlo
    if tag:
        filtered_items = [item for item in items_db if tag in item.tags]
    else:
        filtered_items = items_db
    
    # Aplicamos paginación
    return filtered_items[skip : skip + limit]

# READ - Obtener un item por ID
@app.get(
    "/items/{item_id}",
    response_model=Item
)
def read_item(item_id: int):
    """
    Obtiene un item específico por su ID.
    
    Args:
        item_id: ID del item a obtener
    
    Returns:
        Item: El item solicitado
    
    Raises:
        HTTPException: 404 si el item no existe
    """
    # Buscamos el item en nuestra "base de datos"
    for item in items_db:
        if item.id == item_id:
            return item
    
    # Si no se encuentra, lanzamos una excepción HTTP 404
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item con ID {item_id} no encontrado"
    )

# DELETE - Eliminar un item
@app.delete(
    "/items/{item_id}",
    status_code=status.HTTP_204_NO_CONTENT  # 204 significa eliminación exitosa sin contenido
)
def delete_item(item_id: int):
    """
    Elimina un item por su ID.
    
    Args:
        item_id: ID del item a eliminar
    
    Raises:
        HTTPException: 404 si el item no existe
    """
    global items_db
    
    # Buscamos el índice del item
    for index, item in enumerate(items_db):
        if item.id == item_id:
            # Eliminamos el item de la lista
            items_db.pop(index)
            return  # Con 204 no retornamos contenido
    
    # Si no se encuentra, lanzamos excepción
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item con ID {item_id} no encontrado"
    )
''')

print("Archivo main_crud.py creado exitosamente")

### Probando la API CRUD con POST

Para probar esta API, debes ejecutar en la terminal:

```bash
uvicorn main_crud:app --reload
```

Luego podemos probar los endpoints desde Python:

In [ ]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

print("=" * 60)
print("PROBANDO API CRUD")
print("=" * 60)

# 1. Crear un nuevo item (POST)
print("\n1. CREAR ITEM (POST):")
new_item = {
    "name": "Laptop Gaming",
    "description": "Laptop potente para gaming y desarrollo",
    "price": 1499.99,
    "tax": 21.0,
    "tags": ["electronics", "gaming", "computers"]
}

# Hacemos la petición POST
# Usamos json= para enviar el diccionario como JSON en el body
response = requests.post(f"{BASE_URL}/items/", json=new_item)
print(f"Status Code: {response.status_code}")  # Debería ser 201
created_item = response.json()
print(f"Item creado: {json.dumps(created_item, indent=2)}")
item_id = created_item['id']  # Guardamos el ID para usar después

# 2. Crear otro item
print("\n2. CREAR SEGUNDO ITEM:")
second_item = {
    "name": "Mouse Inalámbrico",
    "description": "Mouse ergonómico",
    "price": 29.99,
    "tags": ["electronics", "accessories"]
}
response = requests.post(f"{BASE_URL}/items/", json=second_item)
print(f"Status Code: {response.status_code}")
print(f"Item creado: {json.dumps(response.json(), indent=2)}")

# 3. Obtener todos los items (GET)
print("\n3. OBTENER TODOS LOS ITEMS (GET):")
response = requests.get(f"{BASE_URL}/items/")
print(f"Status Code: {response.status_code}")
all_items = response.json()
print(f"Total de items: {len(all_items)}")
for item in all_items:
    print(f"  - {item['name']} (ID: {item['id']}) - ${item['price']}")

# 4. Obtener un item específico (GET con ID)
print(f"\n4. OBTENER ITEM POR ID (GET /items/{item_id}):")
response = requests.get(f"{BASE_URL}/items/{item_id}")
print(f"Status Code: {response.status_code}")
print(f"Item: {json.dumps(response.json(), indent=2)}")

# 5. Filtrar items por tag
print("\n5. FILTRAR ITEMS POR TAG 'gaming':")
response = requests.get(f"{BASE_URL}/items/", params={"tag": "gaming"})
print(f"Items con tag 'gaming': {len(response.json())}")
for item in response.json():
    print(f"  - {item['name']}")

# 6. Intentar obtener un item que no existe (debería dar error 404)
print("\n6. INTENTAR OBTENER ITEM INEXISTENTE:")
response = requests.get(f"{BASE_URL}/items/999")
print(f"Status Code: {response.status_code}")  # Debería ser 404
print(f"Error: {response.json()}")

print("\n" + "=" * 60)

---
## 5. Métodos PUT y PATCH para Actualización

### Diferencia entre PUT y PATCH

- **PUT**: Reemplaza el recurso completo. Debes enviar todos los campos.
- **PATCH**: Actualiza parcialmente. Solo envías los campos que quieres cambiar.

Ejemplo:
- PUT: "Reemplaza este usuario con estos nuevos datos completos"
- PATCH: "Solo cambia el email de este usuario"

### Implementando PUT y PATCH

In [ ]:
# Actualizamos main_crud.py para añadir PUT y PATCH

with open('main_crud.py', 'a', encoding='utf-8') as f:
    f.write('''\n
# UPDATE - Actualizar un item completo (PUT)
@app.put(
    "/items/{item_id}",
    response_model=Item
)
def update_item(item_id: int, updated_item: ItemCreate):
    """
    Actualiza un item completo por su ID.
    
    PUT reemplaza el item completo, por lo que debes proporcionar todos los campos.
    
    Args:
        item_id: ID del item a actualizar
        updated_item: Nuevos datos del item
    
    Returns:
        Item: El item actualizado
    
    Raises:
        HTTPException: 404 si el item no existe
    """
    # Buscamos el item
    for index, item in enumerate(items_db):
        if item.id == item_id:
            # Mantenemos el ID y created_at originales
            new_item = Item(
                id=item.id,
                created_at=item.created_at,
                **updated_item.dict()
            )
            # Reemplazamos el item en la lista
            items_db[index] = new_item
            return new_item
    
    # Si no existe, lanzamos error
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item con ID {item_id} no encontrado"
    )

# Modelo para actualización parcial (PATCH)
class ItemUpdate(BaseModel):
    """
    Modelo para actualización parcial de un Item.
    Todos los campos son opcionales.
    """
    name: Optional[str] = Field(None, min_length=1, max_length=100)
    description: Optional[str] = Field(None, max_length=500)
    price: Optional[float] = Field(None, gt=0)
    tax: Optional[float] = Field(None, ge=0, le=100)
    tags: Optional[List[str]] = None

# UPDATE - Actualizar parcialmente un item (PATCH)
@app.patch(
    "/items/{item_id}",
    response_model=Item
)
def partial_update_item(item_id: int, item_update: ItemUpdate):
    """
    Actualiza parcialmente un item.
    
    Solo necesitas enviar los campos que quieres actualizar.
    
    Args:
        item_id: ID del item a actualizar
        item_update: Campos a actualizar
    
    Returns:
        Item: El item actualizado
    
    Raises:
        HTTPException: 404 si el item no existe
    """
    # Buscamos el item
    for index, item in enumerate(items_db):
        if item.id == item_id:
            # Obtenemos solo los campos que fueron proporcionados (no None)
            # exclude_unset=True excluye campos que no fueron enviados
            update_data = item_update.dict(exclude_unset=True)
            
            # Creamos el item actualizado combinando datos antiguos y nuevos
            updated_item = item.copy(update=update_data)
            
            # Actualizamos en la lista
            items_db[index] = updated_item
            return updated_item
    
    # Si no existe, lanzamos error
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item con ID {item_id} no encontrado"
    )
''')

print("Archivo main_crud.py actualizado con PUT y PATCH")

### Probando PUT y PATCH

In [ ]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

print("=" * 60)
print("PROBANDO PUT Y PATCH")
print("=" * 60)

# Primero creamos un item para actualizar
print("\n1. CREAR ITEM INICIAL:")
initial_item = {
    "name": "Teclado Mecánico",
    "description": "Teclado con switches azules",
    "price": 89.99,
    "tax": 21.0,
    "tags": ["electronics", "accessories", "gaming"]
}
response = requests.post(f"{BASE_URL}/items/", json=initial_item)
created = response.json()
item_id = created['id']
print(f"Item creado con ID: {item_id}")
print(f"Datos iniciales: {json.dumps(created, indent=2)}")

# 2. Actualización completa con PUT
print(f"\n2. ACTUALIZACIÓN COMPLETA CON PUT (ID {item_id}):")
complete_update = {
    "name": "Teclado Mecánico RGB",  # Cambiamos el nombre
    "description": "Teclado con switches rojos y RGB",  # Nueva descripción
    "price": 129.99,  # Nuevo precio
    "tax": 21.0,  # Mismo tax
    "tags": ["electronics", "gaming", "rgb"]  # Nuevos tags
}
# PUT requiere todos los campos
response = requests.put(f"{BASE_URL}/items/{item_id}", json=complete_update)
print(f"Status Code: {response.status_code}")
updated = response.json()
print(f"Item actualizado: {json.dumps(updated, indent=2)}")

# 3. Actualización parcial con PATCH - solo precio
print(f"\n3. ACTUALIZACIÓN PARCIAL CON PATCH (solo precio):")
partial_update = {
    "price": 99.99  # Solo actualizamos el precio
}
# PATCH permite enviar solo los campos que queremos cambiar
response = requests.patch(f"{BASE_URL}/items/{item_id}", json=partial_update)
print(f"Status Code: {response.status_code}")
updated = response.json()
print(f"Item actualizado (solo precio cambió): {json.dumps(updated, indent=2)}")

# 4. Actualización parcial con PATCH - múltiples campos
print(f"\n4. ACTUALIZACIÓN PARCIAL CON PATCH (nombre y tags):")
partial_update = {
    "name": "Teclado Mecánico RGB Premium",
    "tags": ["electronics", "gaming", "rgb", "premium"]
}
response = requests.patch(f"{BASE_URL}/items/{item_id}", json=partial_update)
print(f"Status Code: {response.status_code}")
updated = response.json()
print(f"Item actualizado: {json.dumps(updated, indent=2)}")

# 5. Verificamos el estado final
print(f"\n5. VERIFICAR ESTADO FINAL DEL ITEM:")
response = requests.get(f"{BASE_URL}/items/{item_id}")
final_state = response.json()
print(f"Estado final: {json.dumps(final_state, indent=2)}")

# 6. Eliminamos el item
print(f"\n6. ELIMINAR ITEM (DELETE):")
response = requests.delete(f"{BASE_URL}/items/{item_id}")
print(f"Status Code: {response.status_code}")  # Debería ser 204

# 7. Verificamos que fue eliminado
print("\n7. VERIFICAR QUE FUE ELIMINADO:")
response = requests.get(f"{BASE_URL}/items/{item_id}")
print(f"Status Code: {response.status_code}")  # Debería ser 404
if response.status_code == 404:
    print(f"Error (esperado): {response.json()}")

print("\n" + "=" * 60)

---
## 6. Códigos de Estado HTTP (Status Codes)

### ¿Qué son los códigos de estado?

Los códigos de estado HTTP son números de 3 dígitos que el servidor envía en la respuesta para indicar el resultado de la petición.

### Categorías principales:

#### 2xx - Éxito
- **200 OK**: Petición exitosa (GET, PUT, PATCH)
- **201 Created**: Recurso creado exitosamente (POST)
- **204 No Content**: Petición exitosa sin contenido en respuesta (DELETE)

#### 3xx - Redirección
- **301 Moved Permanently**: El recurso se movió permanentemente
- **302 Found**: El recurso se movió temporalmente

#### 4xx - Error del cliente
- **400 Bad Request**: Petición mal formada
- **401 Unauthorized**: No autenticado
- **403 Forbidden**: Autenticado pero sin permisos
- **404 Not Found**: Recurso no encontrado
- **422 Unprocessable Entity**: Error de validación

#### 5xx - Error del servidor
- **500 Internal Server Error**: Error interno del servidor
- **503 Service Unavailable**: Servicio no disponible

### Usando status codes en FastAPI

In [ ]:
# Ejemplo de uso de diferentes status codes

with open('status_codes_example.py', 'w', encoding='utf-8') as f:
    f.write('''from fastapi import FastAPI, HTTPException, status, Response
from pydantic import BaseModel

app = FastAPI(title="Ejemplo de Status Codes")

class Message(BaseModel):
    message: str

# 200 OK (por defecto)
@app.get("/", status_code=status.HTTP_200_OK)
def read_root():
    """Retorna 200 OK por defecto."""
    return {"message": "Hello World"}

# 201 Created
@app.post("/items/", status_code=status.HTTP_201_CREATED)
def create_item(message: Message):
    """Retorna 201 Created cuando se crea un recurso."""
    return {"message": f"Item creado: {message.message}"}

# 204 No Content
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_item(item_id: int):
    """Retorna 204 No Content al eliminar (sin cuerpo de respuesta)."""
    # Con 204, no retornamos nada
    return

# 404 Not Found
@app.get("/items/{item_id}")
def get_item(item_id: int):
    """Retorna 404 si el item no existe."""
    if item_id != 1:  # Solo existe el item 1
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Item no encontrado"
        )
    return {"item_id": item_id, "name": "Item 1"}

# 400 Bad Request
@app.post("/validate/")
def validate_data(value: int):
    """Retorna 400 si los datos son inválidos."""
    if value < 0:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="El valor no puede ser negativo"
        )
    return {"value": value, "valid": True}

# 500 Internal Server Error
@app.get("/error/")
def trigger_error():
    """Simula un error interno del servidor."""
    raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail="Error interno del servidor"
    )

# Status code personalizado basado en lógica
@app.get("/custom-status/")
def custom_status(response: Response, should_succeed: bool = True):
    """
    Ejemplo de cómo cambiar el status code dinámicamente.
    """
    if should_succeed:
        response.status_code = status.HTTP_200_OK
        return {"message": "Operación exitosa"}
    else:
        response.status_code = status.HTTP_400_BAD_REQUEST
        return {"message": "Operación fallida"}
''')

print("Archivo status_codes_example.py creado")

---
## Resumen de la Parte 2

En esta segunda parte del bootcamp hemos aprendido:

### 1. Pydantic y BaseModel
- Qué es Pydantic y por qué es fundamental en FastAPI
- Cómo crear modelos con `BaseModel`
- Validación automática de tipos
- Conversión de modelos a dict y JSON
- Manejo de errores de validación con `ValidationError`

### 2. Validaciones Avanzadas
- Uso de `Field` para añadir restricciones (min_length, max_length, gt, ge, lt, le)
- Validadores personalizados con `@validator`
- Validación de formatos con regex
- Validación de múltiples campos relacionados

### 3. Métodos HTTP
- **GET**: Leer datos
- **POST**: Crear recursos
- **PUT**: Actualizar recursos completos
- **PATCH**: Actualizar recursos parcialmente
- **DELETE**: Eliminar recursos

### 4. Request Body
- Diferencia entre query parameters y request body
- Uso de modelos Pydantic como request body
- Validación automática del body

### 5. API CRUD Completa
- Implementación de Create, Read, Update, Delete
- Uso de `response_model` para especificar el modelo de respuesta
- Manejo de errores con `HTTPException`
- Diferencia entre `ItemCreate` (para entrada) e `Item` (para respuesta)

### 6. Códigos de Estado HTTP
- **200**: OK
- **201**: Created
- **204**: No Content
- **400**: Bad Request
- **404**: Not Found
- **422**: Unprocessable Entity
- **500**: Internal Server Error

### Conceptos clave:
- `exclude_unset=True`: Para PATCH, obtener solo campos enviados
- `.copy(update=...)`: Para actualizar un modelo Pydantic
- `status_code` en decoradores para especificar el código de respuesta
- `HTTPException` para retornar errores con códigos específicos

---

### Próximos Pasos (Parte 3)

En la siguiente parte aprenderemos:
- Integración con bases de datos SQLite
- SQLAlchemy como ORM (Object-Relational Mapping)
- Creación de modelos de base de datos
- Migraciones y gestión de esquemas
- Relaciones entre tablas
- Conexión a la base de datos desde FastAPI
- Operaciones CRUD con base de datos real

---

### Ejercicios Propuestos

Antes de continuar a la Parte 3, practica creando:

1. Un modelo `User` con validaciones personalizadas para email y edad
2. Una API CRUD completa para gestionar usuarios
3. Un endpoint que use PATCH para actualizar solo el email de un usuario
4. Validadores que comprueben que una contraseña tiene al menos una mayúscula y un número

¡Continúa con la Parte 3 cuando estés listo!